In [0]:
df_cust=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,inferSchema=True,sep=',').toDF('id','name','age','city','plan')

df_cust.display()
df_cust.show(10,False)
df_cust.show(10,True)
df_cust.take(5)
display(df_cust.sample(.1))




In [0]:
df_cust.printSchema()
print(df_cust.columns)
print(df_cust.dtypes)

In [0]:
print(df_cust.count())
print(df_cust.distinct().count())
print(df_cust.dropDuplicates().count())
display(df_cust.describe())
display(df_cust.summary())

In [0]:
str1="ide string,name string,age string,city string,plan string"
str2="ids integer,name string,age string,city string,plan string"

df_cust2=spark.read.schema(str2).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,sep=',')

df_cust3=spark.read.schema(str1).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,sep=',')

df_cust4=df_cust2.union(df_cust3)

df_cust4.display()
df_cust4.printSchema()

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType

sch= StructType([StructField('id',IntegerType(),True),StructField('name',StringType(),True),StructField('age',IntegerType(),True),StructField('city',StringType(),True),StructField('plan',StringType(),True)])

df1=spark.read.schema(sch).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,sep=',',mode='permissive')

df2=spark.read.schema(sch).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,sep=',',mode='DROPMALFORMED')

df1.display()
df2.display()
df_cust3.display()


In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType

sch3= StructType([StructField('Id',IntegerType()),StructField('Name',StringType()),StructField('Age',IntegerType()),StructField('City',StringType()),StructField('Plan',StringType()),StructField('CorruptedRecords',StringType())])

df6=spark.read.schema(sch3).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,sep=',',columnNameOfCorruptRecord='CorruptedRecords')

display(df6)




In [0]:
df7=df6.na.drop(how='any')
display(df6)
display(df7)

In [0]:
df8=df6.na.fill('NA',subset=['Plan'])
display(df6)
display(df8)
df6.printSchema()

In [0]:
df8=df6.na.fill({'plan':'noplan','CorruptedRecords':'noCorruptedRecord'})
display(df6)
display(df8)

In [0]:
df9=df6.dropDuplicates(subset=['Plan'])
display(df6)
display(df9)

In [0]:
from pyspark.sql.functions import lit,col,initcap
stddf=df6.withColumn('Source',lit('src')).withColumn('Plan',initcap('plan'))
display(df6)
display(stddf)

In [0]:
df6.where("Name rlike '[0-9]'").show()
stddf=df6.na.fill(30,['Age']).replace({'27':'NA'},'Name')
df6.printSchema()


In [0]:
stddf2=stddf.withColumnRenamed('Id','Cust_id')
display(stddf2)

In [0]:
stddf3=stddf2.select('Cust_id','Name','Age','Plan','City')
display(stddf3)

In [0]:
from pyspark.sql.functions import concat,col,lit

stdf5=stddf3.withColumn('Plan',concat(col('Plan'),lit('-Sim')))

display(stdf5)

In [0]:
from pyspark.sql.functions import split, col

stdf6 = stdf5.withColumn(
    "Plan",
    split(col("Plan"), "-")
)

stdf7 = stdf5.withColumn(
    "Plan",
    split(col("Plan"), "-").getItem(0)
)

display(stdf7)


display(stdf6)


#Data Customization

In [0]:
def upperfn(upperstr):
    return upperstr.upper()

upperudf=udf(upperfn)

stddf9=stdf7.withColumn('City',upperudf('City'))

display(stddf9)


In [0]:
def df_pytthon(Age):
    if Age <18:
        return 'teen'
    elif Age >=18 and Age<=30:
        return 'young'
    elif Age >30 and Age<=50:
        return 'middle'
    else: 
        return 'senior'

In [0]:
from pyspark.sql.functions import udf
df_udf=udf(df_pytthon)

stddf10=stddf9.withColumn('Age',df_udf('Age'))

display(stddf10)

#DATA CURRATION

In [0]:
selectdf=stddf10.select('Cust_id',col('Name').alias('Cust_name'),'Age','City','Plan') #DSL
display(selectdf)

sqlselectdf=stddf10.select('Cust_id',col('Name').alias('Cust_Name'),'Age','City','Plan') #SQL
display(sqlselectdf)

sqlselectdf=stddf10.selectExpr('Cust_id','Name as Cust_Name','Age','City','Plan') #SQL
display(sqlselectdf)

In [0]:
filterdf=stddf9.filter('Cust_id=101')
display(filterdf)

filterdf=stddf9.where('Cust_id=104')
display(filterdf)

In [0]:
casedf=stddf9.selectExpr('*',case when(col('Age')<18,'teen').when(col('Age')>18,'Adult').otherwise('unknown').alias('age_desc'))
display(casedf)